I referred part of the codes & structures from [pytorch-efficientnet-baseline-train-amp-aug]

This notebook is mainly written for my personal practice.



If you have any ideas or questions for this notebook, please feel free to leave a comment :)

In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch
from torch import from_numpy
import torch.nn as nn
import torchvision
from torchvision import models as tvmodels
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.models import resnet
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.cuda.amp import autocast, GradScaler

In [ ]:
#path naming
data_path = "../input/cassava-leaf-disease-classification/"
train_csv_data_path = data_path+"train.csv"
label_json_data_path = data_path+"label_num_to_disease_map.json"
train_img_path = data_path+"train_images/"
test_img_path = data_path+"test_images/"

train_csv = pd.read_csv(train_csv_data_path)
train_csv['label'] = train_csv['label'].astype('string')

# Parameter Setting

In [ ]:
num_class=5

learning_rate = 0.0005
training_epochs = 30
bt_size = 20
num_fold=2
num_wk=4
used_epochs=[5,7,9]

#imagenet mean & std
img_mean = [0.485, 0.456, 0.406]
img_std = [0.229, 0.224, 0.225]

img_size = 224 #to use resnet50

dev = 'cuda:0'

# Data Loading & Transforming

In [ ]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

train = train_csv

In [ ]:
class GetData(Dataset):
    def __init__(self, df, dirr, label_out=True, transform=None):
        super().__init__()
        self.dirr = dirr
        self.label_out = label_out
        self.transform = transform
        self.df = df.reset_index(drop=True).copy()
        if self.label_out == True:
            self.labels = self.df['label'].values
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index:int):
        img = get_img("{}/{}".format(self.dirr, self.df.loc[index]['image_id']))
        if self.label_out == True:
            target = float(self.labels[index])
        
        img = self.transform(image=img)['image']
            
        if self.label_out:    
            return img, target
        if not self.label_out:
            return img
        

In [ ]:
train_trans = A.Compose([
    A.RandomResizedCrop(img_size, img_size),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
    A.Normalize(img_mean, img_std),
    ToTensorV2(),
])

val_trans = A.Compose([
    A.CenterCrop(img_size, img_size),
    A.Normalize(img_mean, img_std),
    ToTensorV2(),
])

test_trans = A.Compose([
    A.CenterCrop(img_size, img_size),
    A.Normalize(img_mean, img_std),
    ToTensorV2(),
])

In [ ]:
def split_load_data(df, train_idx, val_idx, data_root=train_img_path):
    
    
    train_ = df.loc[train_idx,:].reset_index(drop=True)
    valid_ = df.loc[val_idx,:].reset_index(drop=True)
        
    train_ds = GetData(train_, data_root, label_out = True, transform = train_trans)
    valid_ds = GetData(valid_, data_root, label_out = True, transform = val_trans)
    
    train_dl = DataLoader(
        train_ds,
        batch_size=bt_size,
        shuffle=True,        
        num_workers=num_wk,
    )
    val_dl = DataLoader(
        valid_ds, 
        batch_size=bt_size,
        num_workers=num_wk,
        shuffle=False
    )
    return train_dl, val_dl

# NN Model Setting

In [ ]:
def val_one_epoch(epoch, model, loss, data_loader, device):
    model.eval()
    
    preds_all = []
    targets_all = []
    loss_sum = 0
    sample_num = 0
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs, targets) in pbar:
        imgs = imgs.to(device).float()
        targets = targets.to(device).long()
        
        preds = model(imgs)
        preds_all += [torch.argmax(preds, 1).detach().cpu().numpy()]
        targets_all += [targets.detach().cpu().numpy()]
        
        cost = loss(preds, targets)
        
        loss_sum += cost.item()*targets.shape[0]
        sample_num += targets.shape[0]
    
    preds_all = np.concatenate(preds_all)
    targets_all = np.concatenate(targets_all)
    print('accuracy = {:.4f}'.format((preds_all==targets_all).mean()))
    
    return (preds_all==targets_all).mean()


def train_one_epoch(model, loss, optim, data_loader, device):
    model.train()
    scaler = GradScaler()
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs, targets) in pbar:
        imgs = imgs.to(device).float()
        targets = targets.to(device).long()
        optim.zero_grad()
        
        with autocast():
            preds = model(imgs)
            cost = loss(preds, targets)
        
        scaler.scale(cost).backward()
        scaler.step(optim)
        scaler.update()

In [ ]:
def he_init(m):
    if type(m) == nn.Linear:
        nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        m.bias.data.fill_(0.01)

cassava_resnet50 = nn.Sequential(
    resnet.resnet50(pretrained = False),
    nn.Linear(1000, num_class)
)
cassava_resnet50.apply(he_init)

# Training

In [ ]:
device = torch.device(dev)

model = cassava_resnet50.to(device)
loss = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

acc_cnt = []

In [ ]:
folds = StratifiedKFold(n_splits = num_fold).split(np.arange(train.shape[0]), train.label.values)

for fld, (train_idx, val_idx) in enumerate(folds):
    print(len(train_idx), len(val_idx))
    print('Train fold {} started'.format(fld+1))
    
    train_dl, val_dl = split_load_data(train, train_idx, val_idx, data_root=train_img_path)
    
    #starting training
   
    for epochs in range(training_epochs):
        train_one_epoch(model, loss, optimizer, train_dl, device)
    
        with torch.no_grad():
            acc = val_one_epoch(epochs+1, model, loss, val_dl, device)
        acc_cnt.append(acc)
        torch.save(model.state_dict(),'cassnet_{}'.format(training_epochs*fld + epochs))
    
    print('Train finished')

# Prediction

In [ ]:
#Test Dataset & DataLoader
test = pd.DataFrame()
test['image_id'] = list(os.listdir(test_img_path))

test_ds = GetData(test, test_img_path, label_out = False, transform = test_trans)
test_dl = DataLoader(dataset = test_ds, batch_size = bt_size, num_workers = num_wk, shuffle = False)

In [ ]:
acc_cnt = np.array(acc_cnt)
acc_idx = np.argsort(acc_cnt)
acc_idx = acc_idx[::-1]

In [ ]:
use_num = [acc_idx[0], acc_idx[1], acc_idx[2]]
device = torch.device(dev)

model = cassava_resnet50.to(device)

In [ ]:
def infer_one_epoch(model, data_loader, device):
    model.eval()

    preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        preds = model(imgs)
        preds_all += [torch.softmax(preds, 1).detach().cpu().numpy()]
        
    
    preds_all = np.concatenate(preds_all, axis=0)
    return preds_all

In [ ]:
print(use_num)

In [ ]:
test_preds = []

for i in range(3):    
    model.load_state_dict(torch.load('./cassnet_{}'.format(acc_idx[i])))
    with torch.no_grad():
        test_preds += [infer_one_epoch(model, test_dl, device)]   
test_preds = np.mean(test_preds, axis=0)

In [ ]:
test['label'] = np.argmax(test_preds, axis=1)

test.to_csv('submission.csv', index=False)